# Description

Reads the gene pair samples across different categories and computes their p-values.

# Modules loading

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import pandas as pd
from concurrent.futures import as_completed, ProcessPoolExecutor
from tqdm import tqdm

from ccc.coef import ccc
from ccc import conf

# Settings

In [2]:
DATASET_CONFIG = conf.GTEX
GTEX_TISSUE = "whole_blood"
GENE_SEL_STRATEGY = "var_pc_log2"

PVALUE_N_PERMS = 1000000

RANDOM_STATE = np.random.RandomState(0)

# Paths

In [3]:
INPUT_GENE_EXPR_FILE = (
    DATASET_CONFIG["GENE_SELECTION_DIR"]
    / f"gtex_v8_data_{GTEX_TISSUE}-{GENE_SEL_STRATEGY}.pkl"
)
display(INPUT_GENE_EXPR_FILE)

assert INPUT_GENE_EXPR_FILE.exists()

PosixPath('/opt/data/results/gtex_v8/gene_selection/gtex_v8_data_whole_blood-var_pc_log2.pkl')

In [4]:
INPUT_GENE_PAIRS_INTERSECTIONS_FILE = (
    DATASET_CONFIG["GENE_PAIR_INTERSECTIONS"]
    / f"gene_pair_intersections-gtex_v8-{GTEX_TISSUE}-{GENE_SEL_STRATEGY}.pkl"
)
display(INPUT_GENE_PAIRS_INTERSECTIONS_FILE)

assert INPUT_GENE_PAIRS_INTERSECTIONS_FILE.exists()

PosixPath('/opt/data/results/gtex_v8/gene_pair_intersections/gene_pair_intersections-gtex_v8-whole_blood-var_pc_log2.pkl')

In [5]:
OUTPUT_DIR = DATASET_CONFIG["GENE_PAIR_INTERSECTIONS"] / "pvalues"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [6]:
OUTPUT_DIR

PosixPath('/opt/data/results/gtex_v8/gene_pair_intersections/pvalues')

# Load gene expression data

In [7]:
data = pd.read_pickle(INPUT_GENE_EXPR_FILE).sort_index()

In [8]:
data.shape

(5000, 755)

In [9]:
data.head()

,GTEX-111YS-0006-SM-5NQBE,GTEX-1122O-0005-SM-5O99J,GTEX-1128S-0005-SM-5P9HI,GTEX-113IC-0006-SM-5NQ9C,GTEX-113JC-0006-SM-5O997,GTEX-117XS-0005-SM-5PNU6,GTEX-117YW-0005-SM-5NQ8Z,GTEX-1192W-0005-SM-5NQBQ,GTEX-1192X-0005-SM-5NQC3,GTEX-11DXW-0006-SM-5NQ7Y,...,GTEX-ZVE2-0006-SM-51MRW,GTEX-ZVP2-0005-SM-51MRK,GTEX-ZVT2-0005-SM-57WBW,GTEX-ZVT3-0006-SM-51MT9,GTEX-ZVT4-0006-SM-57WB8,GTEX-ZVTK-0006-SM-57WBK,GTEX-ZVZP-0006-SM-51MSW,GTEX-ZVZQ-0006-SM-51MR8,GTEX-ZXES-0005-SM-57WCB,GTEX-ZXG5-0005-SM-57WCN
gene_ens_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419.12,20.6500,25.050,7.155,49.130,6.147,4.1430,5.390,4.389,1.1580,6.8240,...,4.4070,32.340,18.6800,9.251,7.828,7.460,33.2400,5.8480,25.760,17.080
ENSG00000000938.12,906.0000,1344.000,633.500,719.200,392.600,166.5000,338.200,413.200,51.5400,423.6000,...,354.8000,1102.000,774.9000,206.000,620.400,346.300,1304.0000,232.9000,631.600,884.500
ENSG00000001167.14,8.1900,20.010,20.470,21.220,16.460,8.6190,18.220,16.580,1.6020,35.6800,...,11.3400,11.250,11.1800,9.523,41.860,24.580,8.8920,13.3900,13.470,42.640
ENSG00000001561.6,0.7104,1.771,2.234,6.014,3.206,0.3962,2.445,1.418,0.5531,0.7447,...,0.9269,2.555,0.5976,3.417,2.645,1.883,0.5391,0.9816,1.036,6.729
ENSG00000002549.12,22.5000,21.330,19.290,157.100,29.330,9.5770,14.170,23.330,1.4070,28.3000,...,4.4930,50.470,16.2100,32.740,18.150,11.920,20.1000,15.5500,11.980,35.370


# Load gene pairs samples

In [10]:
output_file = OUTPUT_DIR / "gene_pair-samples.pkl"

In [11]:
gene_pair_samples = pd.read_pickle(output_file)

In [12]:
len(gene_pair_samples)

30

In [13]:
sorted(gene_pair_samples.keys())

['all_high-random',
 'all_high-top_ccc',
 'all_high-top_pearson',
 'all_high-top_spearman',
 'all_low-random',
 'all_low-top_ccc',
 'all_low-top_pearson',
 'all_low-top_spearman',
 'ccc_high_and_pearson_low-random',
 'ccc_high_and_pearson_low-top_ccc',
 'ccc_high_and_pearson_low-top_pearson',
 'ccc_high_and_pearson_low-top_spearman',
 'ccc_high_and_spearman_low-top_ccc',
 'ccc_high_and_spearman_low-top_pearson',
 'ccc_high_and_spearman_low-top_spearman',
 'ccc_high_and_spearman_pearson_low-top_ccc',
 'ccc_high_and_spearman_pearson_low-top_pearson',
 'ccc_high_and_spearman_pearson_low-top_spearman',
 'ccc_spearman_high_and_pearson_low-random',
 'ccc_spearman_high_and_pearson_low-top_ccc',
 'ccc_spearman_high_and_pearson_low-top_pearson',
 'ccc_spearman_high_and_pearson_low-top_spearman',
 'entire_dataset-random',
 'pearson_high_and_ccc_low-top_ccc',
 'pearson_high_and_ccc_low-top_pearson',
 'pearson_high_and_ccc_low-top_spearman',
 'pearson_high_and_ccc_spearman_low-top_ccc',
 'pearson_

In [14]:
_k = list(gene_pair_samples.keys())[0]
gene_pair_samples[_k].head()

,,Pearson (high),Pearson (low),Spearman (high),Spearman (low),Clustermatch (high),Clustermatch (low),ccc,pearson,spearman
ENSG00000277089.4,ENSG00000277632.1,True,False,True,False,True,False,0.916918,0.916024,0.955427
ENSG00000171148.13,ENSG00000247596.8,True,False,True,False,True,False,0.896710,0.965141,0.973215
ENSG00000168298.6,ENSG00000124575.6,True,False,True,False,True,False,0.896710,0.869247,0.914341
ENSG00000125910.5,ENSG00000165879.8,True,False,True,False,True,False,0.886690,0.896396,0.940058
ENSG00000130772.13,ENSG00000163154.5,True,False,True,False,True,False,0.876726,0.945764,0.949664


In [15]:
[i for i in gene_pair_samples[_k].head(10).index]

[('ENSG00000277089.4', 'ENSG00000277632.1'),
 ('ENSG00000171148.13', 'ENSG00000247596.8'),
 ('ENSG00000168298.6', 'ENSG00000124575.6'),
 ('ENSG00000125910.5', 'ENSG00000165879.8'),
 ('ENSG00000130772.13', 'ENSG00000163154.5'),
 ('ENSG00000147872.9', 'ENSG00000135245.9'),
 ('ENSG00000197561.6', 'ENSG00000172232.9'),
 ('ENSG00000235568.6', 'ENSG00000181631.6'),
 ('ENSG00000124216.3', 'ENSG00000113070.7'),
 ('ENSG00000146094.13', 'ENSG00000248996.1')]

# Compute pvalues on sampled gene pairs

In [16]:
output_file = OUTPUT_DIR / "gene_pair-samples-pvalues.pkl"

In [17]:
def corr_single(x, y):
    ccc_val, ccc_pval = ccc(x, y, pvalue_n_perms=PVALUE_N_PERMS, n_jobs=1)
    p_val, p_pval = stats.pearsonr(x, y)
    s_val, s_pval = stats.spearmanr(x, y)

    return ccc_val, ccc_pval, p_val, p_pval, s_val, s_pval

In [18]:
results = []

# I leave the ProcessPoolExecutor here in case I want to easily swith between
# parallelize across gene pairs (max_workers=conf.GENERAL["N_JOBS"] and n_jobs=1 inside function corr_single)
# or across permutations for one gene pair (max_workers=1 and n_jobs=conf.GENERAL["N_JOBS"])
with ProcessPoolExecutor(max_workers=conf.GENERAL["N_JOBS"]) as executor:
    tasks = {
        executor.submit(corr_single, data.loc[gene0], data.loc[gene1]): (
            gene0,
            gene1,
            k,
        )
        for k, v in gene_pair_samples.items()
        for gene0, gene1 in gene_pair_samples[k].index
    }

    for t_idx, t in tqdm(enumerate(as_completed(tasks)), total=len(tasks), ncols=100):
        gene0, gene1, k = tasks[t]
        ccc_val, ccc_pval, p_val, p_pval, s_val, s_pval = t.result()

        results.append(
            {
                "gene0": gene0,
                "gene1": gene1,
                "group": k,
                "ccc": ccc_val,
                "ccc_pvalue": ccc_pval,
                "pearson": p_val,
                "pearson_pvalue": p_pval,
                "spearman": s_val,
                "spearman_pvalue": s_pval,
            }
        )

        # save
        _df = pd.DataFrame(results)
        _df["group"] = _df["group"].astype("category")
        _df.to_pickle(output_file)

100%|██████████████████████████████████████████████████████| 12116/12116 [87:00:13<00:00, 25.85s/it]


In [19]:
len(results)

12116

In [20]:
results_df = pd.DataFrame(results)
results_df["group"] = results_df["group"].astype("category")

In [21]:
results_df.shape

(12116, 9)

In [22]:
results_df.head()

,gene0,gene1,group,ccc,ccc_pvalue,pearson,pearson_pvalue,spearman,spearman_pvalue
0,ENSG00000197324.8,ENSG00000171302.16,all_high-top_ccc,0.866819,9.999990e-07,0.958808,0.000000e+00,0.972106,0.000000e+00
1,ENSG00000197561.6,ENSG00000172232.9,all_high-top_ccc,0.876726,9.999990e-07,0.953421,0.000000e+00,0.978590,0.000000e+00
2,ENSG00000147872.9,ENSG00000135245.9,all_high-top_ccc,0.876726,9.999990e-07,0.849159,5.792244e-211,0.911077,5.115786e-292
3,ENSG00000235568.6,ENSG00000169403.11,all_high-top_ccc,0.866819,9.999990e-07,0.934582,0.000000e+00,0.951794,0.000000e+00
4,ENSG00000125910.5,ENSG00000165879.8,all_high-top_ccc,0.886690,9.999990e-07,0.896396,2.758126e-268,0.940058,0.000000e+00


# Remove duplicated gene pairs

This could happen when gene pairs overlap and are the top of different coefficients.

In [23]:
# results_df = results_df.drop_duplicates(subset=["gene0", "gene1"])

In [24]:
# results.shape

# Save

In [25]:
results_df.to_pickle(output_file)